<a href="https://colab.research.google.com/github/Maagnitude/wikimovieplot-nlp-model/blob/main/wikimovieplot-nlp-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3η Εργασία** στο μάθημα **Μηχανική Μάθηση και Εφαρμογές**

# **Τμήμα Πληροφορικής και Τηλεματικής - Χαροκόπειο Πανεπιστήμιο**

# **Καζάζης Γεώργιος - it214124**

Στην παρούσα εργασία θα χρησιμοποιήσουμε μεθόδους επεξεργασίας φυσικής γλώσσας.

In [1]:
import pandas as pd
import numpy as np
import os
import re
import shutil
import string

import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
url = "https://raw.githubusercontent.com/Maagnitude/wikimovieplot-nlp-model/main/dataset.csv"
# df = pd.read_csv(url)
# df = df.sample(frac=1, random_sra)
# df.head()

# url = "https://github.com/Maagnitude/wikimovieplot-nlp-model/blob/main/dataset.csv.gz"
# dataset = tf.keras.utils.get_file("dataset.csv.gz", url, untar=True, cache_dir='.', cache_subdir='')
# dataset_dir = os.path.join(os.path.dirname(dataset), 'dataset')
# os.listdir(dataset_dir)
# train_dir = os.path.join(dataset_dir, 'train')
# os.listdir(train_dir)


In [3]:
def preprocess_text(text):
    text = text.lower() # convert text to lowercase
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\s+', ' ', text)
    return text

In [238]:
df = pd.read_csv(url)
df = df.sample(frac=1, random_state=42)

origin_dummies = pd.get_dummies(df['Origin/Ethnicity'])
genre_dummies = pd.get_dummies(df['Genre'])

genre_dummies.head()
n_labels = genre_dummies.shape[1]

df['Title'] = df['Title'].apply(preprocess_text)
df['Plot'] = df['Plot'].apply(preprocess_text)

df = df.drop('Origin/Ethnicity', axis='columns')
df = df.drop('Genre', axis='columns')
df = pd.concat([df, origin_dummies, genre_dummies], axis=1)

In [244]:
def load_data_wiki(file_path, batch_size=32, p_train=0.65, p_val=0.15):
    
    df = pd.read_csv(url)
    df = df.sample(frac=1, random_state=42)

    n = df.shape[0]
    n_train = int(n * p_train)
    n_val = int(n * p_val)
    n_test = n - n_train - n_val

    df_train, df_val, df_test = df.iloc[:n_train], df.iloc[n_train:n_train+n_val],\
                                                          df.iloc[n_train+n_val:]

    origin_dummies = pd.get_dummies(df_train['Origin/Ethnicity'])
    genre_dummies1 = pd.get_dummies(df_train['Genre'])

    n_labels = genre_dummies.shape[1]

    origin_dummies = pd.get_dummies(df_val['Origin/Ethnicity'])
    genre_dummies2 = pd.get_dummies(df_val['Genre'])

    origin_dummies = pd.get_dummies(df_test['Origin/Ethnicity'])
    genre_dummies3 = pd.get_dummies(df_test['Genre'])

    df['Title'] = df['Title'].apply(preprocess_text)
    df['Plot'] = df['Plot'].apply(preprocess_text)

    df = df.drop('Origin/Ethnicity', axis='columns')
    df = df.drop('Genre', axis='columns')
    df = pd.concat([df, origin_dummies, genre_dummies], axis=1)

    

    
                                                          
                                    
    train_data = {'Title': df_train['Title'].values,
                  'Genre': genre_dummies1.values.astype(np.float64)}

    val_data = {'Title': df_val['Title'].values,
                'Genre': genre_dummies2.values.astype(np.float64)}            

    test_data = {'Title': df_test['Title'].values,
                 'Genre': genre_dummies3.values.astype(np.float64)}

    raw_train_ds = tf.data.Dataset.from_tensor_slices(train_data).batch(batch_size)
    raw_val_ds = tf.data.Dataset.from_tensor_slices(val_data).batch(batch_size)
    raw_test_ds = tf.data.Dataset.from_tensor_slices(test_data).batch(batch_size)

    return raw_train_ds, raw_val_ds, raw_test_ds, n_labels

In [245]:
raw_train_ds, raw_val_ds, raw_test_ds, n_labels = load_data_wiki(url)

In [54]:
n_labels

20

In [263]:
for element in raw_train_ds.take(1):
  print(element['Genre'][1])

tf.Tensor([0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(20,), dtype=float64)


In [264]:
max_features = 500

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary')

In [265]:
def vectorize_text(text):
  return vectorize_layer(tf.expand_dims(text, -1))

In [266]:
train_titles = raw_train_ds.map(lambda x: x['Title'])
train_genre = raw_train_ds.map(lambda y: y['Genre'])
vectorize_layer.adapt(train_titles, train_genre)

In [267]:
val_titles = raw_val_ds.map(lambda x: x['Title'])
val_genre = raw_val_ds.map(lambda y: y['Genre'])
vectorize_layer.adapt(val_titles, val_genre)

In [268]:
test_titles = raw_test_ds.map(lambda x: x['Title'])
test_genre = raw_test_ds.map(lambda y: y['Genre'])
vectorize_layer.adapt(test_titles, test_genre)

In [269]:
train_ds = raw_train_ds.map(lambda x: (vectorize_text(x['Title']), x['Genre']))
val_ds = raw_val_ds.map(lambda x: (vectorize_text(x['Title']), x['Genre']))
test_ds = raw_test_ds.map(lambda x: (vectorize_text(x['Title']), x['Genre']))

In [261]:
train_ds.get_single_element

<bound method DatasetV2.get_single_element of <MapDataset element_spec=(TensorSpec(shape=(None, 500), dtype=tf.float32, name=None), TensorSpec(shape=(None, 20), dtype=tf.float64, name=None))>>

In [273]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 20)                10020     
                                                                 
Total params: 10,020
Trainable params: 10,020
Non-trainable params: 0
_________________________________________________________________


In [275]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/10
468/468 [==============================] - 3s 4ms/step - loss: 0.2894 - accuracy: 0.2814 - val_loss: 0.2381 - val_accuracy: 0.2688
Epoch 2/10
468/468 [==============================] - 2s 4ms/step - loss: 0.2114 - accuracy: 0.2842 - val_loss: 0.1959 - val_accuracy: 0.2716
Epoch 3/10
468/468 [==============================] - 2s 4ms/step - loss: 0.1837 - accuracy: 0.2924 - val_loss: 0.1787 - val_accuracy: 0.2777
Epoch 4/10
468/468 [==============================] - 2s 3ms/step - loss: 0.1712 - accuracy: 0.3013 - val_loss: 0.1704 - val_accuracy: 0.2879
Epoch 5/10
468/468 [==============================] - 3s 6ms/step - loss: 0.1647 - accuracy: 0.3078 - val_loss: 0.1661 - val_accuracy: 0.2922
Epoch 6/10
468/468 [==============================] - 2s 3ms/step - loss: 0.1609 - accuracy: 0.3148 - val_loss: 0.1636 - val_accuracy: 0.2948
Epoch 7/10
468/468 [==============================] - 2s 4ms/step - loss: 0.1585 - accuracy: 0.3219 - val_loss: 0.1620 - val_accuracy: 0.2983
Epoch 